In [1]:
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
import os

In [3]:
subject='sub-001'
MAIN_PATH=os.getenv('DERIVATIVES')
denoising='pybest'
depth='GM'

In [68]:
#In case of laminar analysis, we import outputs from pybest in the func space as giftis and have to calculate psc ourselves for each layer. If outputs from pybest are in .npy format, we can run fitprf (master -m 17) from linescanning pipeline and get the psc calculation from the derivatives/prf folder. Therefore, we here reproduce the psc calculation, first in an usual case (non-laminar) where we do have the outputs in the fsnative space as .npy. NOTE: this has been previously checked and will be removed.

pybest_tc_L = []
pybest_tc_R = []
pybest_tc = []

#Get pybest outputs as .npy files and calculate psc:
for run in range(4):
    pybest_tc_L = np.load(
        f'{MAIN_PATH}/pybest/{subject}/ses-1/unzscored/{subject}_ses-1_task-ret_run-{run + 1}_space-fsnative_hemi-L_desc-denoised_bold.npy')
    pybest_tc_R = np.load(
        f'{MAIN_PATH}/pybest/{subject}/ses-1/unzscored/{subject}_ses-1_task-ret_run-{run + 1}_space-fsnative_hemi-R_desc-denoised_bold.npy')
    tc = np.concatenate([pybest_tc_L, pybest_tc_R], axis=1)
    tc_m = tc * np.expand_dims(np.nan_to_num((100 / np.mean(tc, axis=0))), axis=0)
    baseline = np.median(tc_m[0:19], axis=0)
    tc_m = tc_m - baseline
    pybest_tc.append(tc_m)

mean_pybest_tc = np.median(np.array(pybest_tc), axis=0)
psc = (mean_pybest_tc)

In [6]:
path=f'{MAIN_PATH}/pRFM/{subject}/ses-1/{denoising}/'
os.makedirs(path, exist_ok=True)
#np.save("./pilot1-fsnativeWNORDIC_lamGM/fast_fit/sub-001_ses-1_task-ret_hemi-LR_desc-avg_bold_lamGM.npy",psc)
np.save(f'{MAIN_PATH}/pRFM/{subject}/ses-1/{denoising}/{subject}_ses-1_task-ret_hemi-LR_desc-avg_bold_{depth}.npy',psc)